# Lecture 17 – Regular Expressions

## DSC 80, Winter 2023

### 📣 Announcements
- Lab 6 (web scraping and APIs) is due **today at 4:00PM (no slip days!).**
    - No slip days are allowed because we will take up the solutions in Discussion 6 at 5:00PM today.
- Project 3 is due **tomorrow at 11:59PM**.
- Midterm Exam scores are released, and regrades are due **tonight at 11:59PM**.
    - Remember that it's only worth 10%, and that we have a redemption policy.
- Lab 7 (regular expressions and text features) is due on **Monday, February 27th at 11:59PM**.
- Followup from last class: `BeautifulSoup` objects are mutable! See [this post on Ed by Trey](https://edstem.org/us/courses/32057/discussion/2615896) for more details.

### Agenda

Lots and lots of regular expressions! Good resources:
- [regex101.com](https://regex101.com), a helpful site to have open while writing regular expressions.
- Python [`re` library documentation](https://docs.python.org/3/library/re.html) and [how-to](https://docs.python.org/3/howto/regex.html).
    - The "how-to" is great, read it!
- [regex "cheat sheet"](https://dsc80.com/resources/other/berkeley-regex-reference.pdf) (taken from [here](https://ds100.org/sp22/resources/)).

See [dsc80.com/resources/#regular-expressions](https://dsc80.com/resources/#regular-expressions).

## Motivation

In [1]:
contact = '''
Thank you for buying our expensive product!

If you have a complaint, please send it to complaints@compuserve.com or call (800) 867-5309.

If you are happy with your purchase, please call us at (800) 123-4567; we'd love to hear from you!

Due to high demand, please allow one-hundred (100) business days for a response.
'''

### Who called? 📞

- **Goal**: Extract all phone numbers from a piece of text, assuming they are of the form `'(###) ###-####'`.

In [2]:
print(contact)


Thank you for buying our expensive product!

If you have a complaint, please send it to complaints@compuserve.com or call (800) 867-5309.

If you are happy with your purchase, please call us at (800) 123-4567; we'd love to hear from you!

Due to high demand, please allow one-hundred (100) business days for a response.



- We can do this using the same string methods we've come to know and love.

- Strategy:
    - Split by spaces.
    - Check if there are any consecutive "words" where:
        - the first "word" looks like an area code, like `'(678)'`.
        - the second "word" looks like the last 7 digits of a phone number, like `'999-8212'`. 

Let's first write a function that takes in a string and returns whether it looks like an area code.

In [3]:
def is_possibly_area_code(s):
    '''Does `s` look like (678)?'''
    return len(s) == 5 and s.startswith('(') and s.endswith(')') and s[1:4].isnumeric()

In [4]:
is_possibly_area_code('(123)')

True

In [5]:
is_possibly_area_code('(99)')

False

Let's also write a function that takes in a string and returns whether it looks like the last 7 digits of a phone number.

In [6]:
def is_last_7_phone_number(s):
    '''Does `s` look like 999-8212?'''
    return len(s) == 8 and s[0:3].isnumeric() and s[3] == '-' and s[4:].isnumeric()

In [7]:
is_last_7_phone_number('999-8212')

True

In [8]:
is_last_7_phone_number('534 1100')

False

Finally, let's split the entire text by spaces, and check whether there are any instances where `pieces[i]` looks like an area code and `pieces[i+1]` looks like the last 7 digits of a phone number.

In [9]:
# Removes punctuation from the end of each string.
pieces = [s.rstrip('.,?;"\'') for s in contact.split()]

for i in range(len(pieces) - 1):
    if is_possibly_area_code(pieces[i]):
        if is_last_7_phone_number(pieces[i+1]):
            print(pieces[i], pieces[i+1])

(800) 867-5309
(800) 123-4567


### Is there a better way?

- This was an example of **pattern matching**.

- It can be done with string methods, but there is often a better approach: **regular expressions**.

In [10]:
print(contact)


Thank you for buying our expensive product!

If you have a complaint, please send it to complaints@compuserve.com or call (800) 867-5309.

If you are happy with your purchase, please call us at (800) 123-4567; we'd love to hear from you!

Due to high demand, please allow one-hundred (100) business days for a response.



In [11]:
import re
re.findall(r'\(\d{3}\) \d{3}-\d{4}', contact)

['(800) 867-5309', '(800) 123-4567']

<center><h3>🤯</h3></center>

## Basic regular expressions

### Regular expressions

- A regular expression, or **regex** for short, is a sequence of characters used to **match patterns in strings**.
    - For example, `\(\d{3}\) \d{3}-\d{4}` describes a **pattern** that matches US phone numbers of the form `'(XXX) XXX-XXXX'`.
    - Think of regex as a "mini-language" (formally: they are a grammar for describing a language).

- **Pros**: They are very powerful and are widely used (virtually every programming language has a module for working with them).

- **Cons**: They can be hard to read and have many different "dialects."

### Writing regular expressions

- You will ultimately write most of your regular expressions in Python, using the `re` module. We will see how to do so shortly.

- However, a useful tool for designing regular expressions is [regex101.com](https://regex101.com).

- We will use it heavily during lecture; you should have it open as we work through examples. **If you're trying to revisit this lecture in the future, you'll likely want to watch the podcast.**

### Literals

- A literal is a character that has no special meaning.

- Letters, numbers, and some symbols are all literals.

- Some symbols, like `.`, `*`, `(`, and `)`, are special characters.

- ***Example:*** The regex `hey` matches the string `'hey'`. The regex `he.` also matches the string `'hey'`.

### Regex building blocks 🧱

The four main building blocks for all regexes are shown below ([table source](https://www.cs.princeton.edu/courses/archive/spring17/cos226/lectures/54RegularExpressions.pdf), [inspiration](https://docs.google.com/presentation/d/1xQsqa7e3xDZ9nBiekbSBOecwvQm8pSVGa-FBoV6aJ7E/edit#slide=id.g11197671c7e_0_919)).

| operation | order of op. | example | matches ✅ | does not match ❌ |
|:--- |:---|:---|:---|:---|
| <span style='color:purple'><b>concatenation</b></span> | 3 | `AABAAB` | `'AABAAB'` | every other string |
| <span style='color:purple'><b>or</b></span> | 4 | `AA\|BAAB` | `'AA'`, `'BAAB'` | every other string |
| <span style='color:purple'><b>closure</b><br>(zero or more)</span> | 2 | `AB*A` | `'AA'`, `'ABBBBBBA'` | `'AB'`, `'ABABA'` |
| <span style='color:purple'><b>parentheses</b></span> | 1 | `A(A\|B)AAB` <hr style="height:1px"> `(AB)*A` | `'AAAAB'`, `'ABAAB'`<hr style="height:1px">`'A'`, `'ABABABABA'` | every other string<hr style="height:1px">`'AA'`, `'ABBA'` |

Note that `|`, `(`, `)`, and `*` are **special characters**, not literals. They manipulate the characters around them.

***Example (or, parentheses):*** 
- What does `DSC 30|80` match?
- What does `DSC (30|80)` match?

***Example (closure, parentheses):*** 
- What does `blah*` match?
- What does `(blah)*` match?

### Exercise

Write a regular expression that matches `'billy'`, `'billlly'`, `'billlllly'`, etc.
- First, think about how to match strings with any even number of `'l'`s, including zero `'l'`s (i.e. `'biy'`).
- Then, think about how to match only strings with a **positive even** number of `'l'`s.

<br><br>

<details>
<summary>
    ✅ Click here to see the answer <b>after</b> you've tried it yourself at <a href='https://regex101.com'>regex101.com</a>.
</summary>
<code>bi(ll)*y</code> will match any even number of <code>'l'</code>s, including 0.
    
To match only a positive even number of <code>'l'</code>s, we'd need to first "fix into place" two <code>'l'</code>s, and then follow that up with zero or more pairs of <code>'l'</code>s. This specifies the regular expression <code>bill(ll)*y</code>.
    </details>

### Exercise

Write a regular expression that matches `'billy'`, `'billlly'`, `'biggy'`, `'biggggy'`, etc.

Specifically, it should match any string with a **positive even** number of `'l'`s in the middle, or a **positive even** number of `'g'`s in the middle.

<br>

<details>
<summary>
    ✅ Click here to see the answer <b>after</b> you've tried it yourself at <a href='https://regex101.com'>regex101.com</a>.
</summary>

Possible answers: <code>bi(ll(ll)\*|gg(gg)\*)y</code> or <code>bill(ll)\*y|bigg(gg)\*y</code>.
 
<br>

Note, <code>bill(ll)\*|gg(gg)\*y</code> is <b>not</b> a valid answer! This is because "concatenation" comes before "or" in the order of operations. This regular expression would match strings that match <code>bill(ll)\*</code>, like <code>'billll'</code>, OR strings that match <code>gg(gg)\*y</code>, like <code>'ggy'</code>.

    
</details>

## Intermediate regex

### More regex syntax

| operation | example | matches ✅ | does not match ❌ |
|:--- |:---|:---|:---|
| <span style='color:purple'><b>wildcard</b></span> | `.U.U.U.` | `'CUMULUS'`<br>`'JUGULUM'` | `'SUCCUBUS'`<br>`'TUMULTUOUS'` |
| <span style='color:purple'><b>character class</b></span>  | `[A-Za-z][a-z]*` | `'word'`<br>`'Capitalized'` | `'camelCase'`<br>`'4illegal'` |
| <span style='color:purple'><b>at least one</b></span> | `bi(ll)+y` | `'billy'`<br>`'billlllly'` | `'biy'`<br>`'bily'` |
| <span style='color:purple'><b>between $i$ and $j$ occurrences</b></span> | `m[aeiou]{1,2}m` | `'mem'`<br>`'maam'`<br>`'miem'` | `'mm'`<br>`'mooom'`<br>`'meme'` |

`.`, `[`, `]`, `+`, `{`, and `}` are also special characters, in addition to `|`, `(`, `)`, and `*`.

***Example (character classes, at least one):*** `[A-E]+` is just shortform for `(A|B|C|D|E)(A|B|C|D|E)*`.

***Example (wildcard):*** 
- What does `.` match? 
- What does `he.` match? 
- What does `...` match?

***Example (at least one, closure):*** 
- What does `123+` match?
- What does `123*` match?

***Example (number of occurrences):*** What does `tri{3, 5}` match? Does it match `'triiiii'`?

***Example (character classes, number of occurrences):*** What does `[1-6a-f]{3}-[7-9E-S]{2}` match?

### Exercise

Write a regular expression that matches any lowercase string has a repeated vowel, such as `'noon'`, `'peel'`, `'festoon'`, or `'zeebraa'`.

<br>

<details>
<summary>
    ✅ Click here to see the answer <b>after</b> you've tried it yourself at <a href='https://regex101.com'>regex101.com</a>.
</summary>

One answer: <code>[a-z]\*(aa|ee|ii|oo|uu)[a-z]\*</code>
 
<br>

This regular expression matches strings of lowercase characters that have <code>'aa'</code>, <code>'ee'</code>, <code>'ii'</code>, <code>'oo'</code>, or <code>'uu'</code> in them anywhere. <code>[a-z]\*</code> means "zero or more of any lowercase characters"; essentially we are saying it doesn't matter what letters come before or after the double vowels, as long as the double vowels exist somewhere.

    
</details>

### Exercise

Write a regular expression that matches any string that contains **both** a lowercase letter and a number, in any order. Examples include `'billy80'`, `'80!!billy'`, and `'bil8ly0'`.

<br>

<details>
<summary>
    ✅ Click here to see the answer <b>after</b> you've tried it yourself at <a href='https://regex101.com'>regex101.com</a>.
</summary>

One answer: <code>(.\*[a-z].\*[0-9].\*)|(.\*[0-9].\*[a-z].\*)</code>
 
<br>

We can break the above regex into two parts – everything before the `|`, and everything after the `|`.

The first part, <code>.\*[a-z].\*[0-9].\*</code>, matches strings in which there is at least one lowercase character and at least one digit, with the lowercase character coming first.

The second part, <code>.\*[0-9].\*[a-z].\*</code>, matches strings in which there is at least one lowercase character and at least one digit, with the digit coming first.
    
Note, the <code>.\*</code> between the digit and letter classes is needed in the event the string has non-digit and non-letter characters.
    
<b>This is the kind of task that would be easier to accomplish with regular Python string methods.</b>

    
</details>

### Even more regex syntax

| operation | example | matches ✅ | does not match ❌ |
|:--- |:---|:---|:---|
| <span style='color:purple'><b>escape character</b></span> | `ucsd\.edu` | `'ucsd.edu'` | `'ucsd!edu'` |
| <span style='color:purple'><b>beginning of line</b></span> | `^ark` | `'ark two'`<br>`'ark o ark'` | `'dark'` |
| <span style='color:purple'><b>end of line</b></span>  | `ark$` | `'dark'`<br>`'ark o ark'` | `'ark two'` |
| <span style='color:purple'><b>zero or one</b></span> | `cat?` | `'ca'`<br>`'cat'` | `'cart'` (matches `'ca'` only) |
| <span style='color:purple'><b>built-in character classes*</b></span> | `\w+` <br> `\d+` | `'billy'`<br>`'231231'` | `'this person'`<br>`'858 people'` |
| <span style='color:purple'><b>character class negation</b></span> | `[^a-z]+` | `'KINGTRITON551'`<br>`'1721$$'` | `'porch'`<br>`'billy.edu'` |

****Note: in Python's implementation of regex,*** 
- `\d` refers to digits.
- `\w` refers to alphanumeric characters (`[A-Z][a-z][0-9]_`).
- `\s` refers to whitespace.
- `\b` is a word boundary.

***Example (escaping):*** 
- What does `he.` match? 
- What does `he\.` match? 
- What does `(858)` match? 
- What does `\(858\)` match?

***Example (anchors):*** 
- What does `858-534` match?
- What does `^858-534` match?
- What does `858-534$` match?

### Example (built-in character classes)

****Note: in Python's implementation of regex,*** 
- `\d` refers to digits.
- `\w` refers to alphanumeric characters (`[A-Z][a-z][0-9]_`).
- `\s` refers to whitespace.
- `\b` is a word boundary.

- What does `\d{3} \d{3}-\d{4}` match?
- What does `\bcat\b` match? Does it find a match in `'my cat is hungry'`? What about `'concatenate'`?

### Exercise

Write a regular expression that matches any string that:
- is between 5 and 10 characters long, and
- is made up of only vowels (either uppercase or lowercase, including `'Y'` and `'y'`), periods, and spaces.

Examples include `'yoo.ee.IOU'` and `'AI.I oey'`.

<br>

<details>
<summary>
    ✅ Click here to see the answer <b>after</b> you've tried it yourself at <a href='https://regex101.com'>regex101.com</a>.
</summary>

One answer: <code>^[aeiouyAEIOUY. ]{5,10}$</code>
 
<br>

<b>Key idea:</b> Within a character class (i.e. <code>[...]</code>), special characters do not generally need to be escaped.


    
</details>

## Regex in Python

### `re` in Python

The `re` package is built into Python. It allows us to use regular expressions to find, extract, and replace strings.

In [12]:
import re

`re.search` takes in a string `regex` and a string `text` and returns the location and substring corresponding to the **first** match of `regex` in `text`.

In [13]:
re.search('AB*A', 
          'here is a string for you: ABBBA. here is another: ABBBBBBBA')

<re.Match object; span=(26, 31), match='ABBBA'>

`re.findall` takes in a string `regex` and a string `text` and returns a list of all matches of `regex` in `text`. You'll use this most often.

In [14]:
re.findall('AB*A', 
           'here is a string for you: ABBBA. here is another: ABBBBBBBA')

['ABBBA', 'ABBBBBBBA']

`re.sub` takes in a string `regex`, a string `repl`, and a string `text`, and replaces all matches of `regex` in `text` with `repl`.

In [15]:
re.sub('AB*A', 
       'billy', 
       'here is a string for you: ABBBA. here is another: ABBBBBBBA')

'here is a string for you: billy. here is another: billy'

### Raw strings

When using regular expressions in Python, it's a good idea to use **raw strings**, denoted by an `r` before the quotes, e.g. `r'exp'`.

In [16]:
re.findall('\bcat\b', 'my cat is hungry')

[]

In [17]:
re.findall(r'\bcat\b', 'my cat is hungry')

['cat']

In [18]:
# Huh?
print('\bcat\b')

cat


### Capture groups
* Surround a regex with `(` and `)` to define a **capture group** within a pattern.
- Capture groups are useful for extracting relevant parts of a string.

In [19]:
re.findall(r'\w+@(\w+)\.edu', 
           'my old email was billy@notucsd.edu, my new email is notbilly@ucsd.edu')

['notucsd', 'ucsd']

- Notice what happens if we remove the `(` and `)`!

In [20]:
re.findall(r'\w+@\w+\.edu', 
           'my old email was billy@notucsd.edu, my new email is notbilly@ucsd.edu')

['billy@notucsd.edu', 'notbilly@ucsd.edu']

- Earlier, we also saw that parentheses can be used to group parts of a regex together. When using `re.findall`, all groups are treated as capturing groups.

In [21]:
# A regex that matches strings with two of the same vowel followed by 3 digits
# We only want to capture the digits, but...
re.findall(r'(aa|ee|ii|oo|uu)(\d{3})', 'eeoo124')

[('oo', '124')]

## Summary, next time

### Summary

- Regular expressions are used to match and extract patterns from text.
- You don't need to force yourself to "memorize" regex syntax – refer to the resources in the [Agenda](#Agenda) section of the lecture and on the [Resources](https://dsc80.com/resources#regular-expressions) tab of the course website.
- Also refer to the three tables of syntax in the lecture:
    - [Regex building blocks](#Regex-building-blocks-🧱).
    - [More regex syntax](#More-regex-syntax).
    - [Even more regex syntax](#Even-more-regex-syntax).
- **Note:** You don't always have to use regular expressions! If Python/`pandas` string methods work for your task, you can still use those.
- **Play [Regex Golf](https://alf.nu/RegexGolf?world=regex&level=r00) to practice!** 🏌️

### Next time

- A few more examples of regular expressions.
- Using regular expressions in `pandas` (through `.str`). 
- Describing text data quantitatively.